### Подключение

Первым делом нужно подключиться к кластеру Spark своей команды (см. инструкцию по заведению кластера). Он запущен на каком-то кластере YT и у него есть какой-то id, их надо указать в конфиге ~/spyt.yaml или при подключении:

In [2]:
!cat ~/spyt.yaml

yt_proxy: "hume"
discovery_path: "//home/sashbel/spark-test"


In [3]:
import spyt
spark = spyt.connect(yt_proxy="hume", 
                     discovery_path="//home/sashbel/spark-test")

2020-10-14 12:39:33,541 - INFO - spyt.client - SPYT Cluster version: #3.0.1-1.1.0+yandex
2020-10-14 12:39:33,545 - INFO - spyt.client - SPYT library version: 1.1.0


In [4]:
spyt.info(spark)

In [21]:
import pyspark.sql.functions as f
import pyspark.sql.types as t

### Чтение и базовые операции

Объект spark -- это SparkSession, точка входа для API. У него есть несколько полезных методов, самый интересный -- метод read. Давайте прочитаем какую-нибудь таблицу с YT.

In [4]:
df = spark.read.yt("//sys/spark/examples/example_1")

Результат этого метода называется DataFrame. Это такая табличка со строгой схемой, которую спарк знает, как посчитать. На самом деле сейчас он прочитал из YT только схему, данные пока не читались. Схему датафрейма можно вывести в любой момент, это не триггерит никаких вычислений

In [5]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- uuid: string (nullable = true)



У датафрейма есть метод show, который печатает его первые 20 строк. Его вызов уже триггерит вычисление: нужно сходить в YT и прочитать небольшую часть таблицы.

In [6]:
df.show()

+---+--------------------+
| id|                uuid|
+---+--------------------+
|  1|bdc3aa61-cf2e-435...|
|  2|79f9566a-b9fc-44b...|
|  3|9f8ea8a8-97cd-44b...|
|  4|3dca3b34-5210-4aa...|
|  5|65d97e4d-21f2-449...|
|  6|cee5a7ec-1962-40e...|
|  7|9a5088bc-21f3-4e1...|
|  8|c3a1d801-35f0-45f...|
|  9|f9c8c91b-24db-48d...|
| 10|3535e360-9244-46f...|
| 11|2e5c9611-a228-4e8...|
| 12|6efca41f-7b13-4f1...|
| 13|4c4ff81e-d5c1-4ec...|
| 14|7e3368f3-64cf-4af...|
| 15|3e250454-5a3d-473...|
| 16|48b89381-4eaa-49d...|
| 17|3fbeebb5-e13e-412...|
| 18|68ffe6c3-2096-43f...|
| 19|ba33e170-32a9-459...|
| 20|4a192a1e-d79e-459...|
+---+--------------------+
only showing top 20 rows



In [7]:
df.show(10, truncate=False)

+---+------------------------------------+
|id |uuid                                |
+---+------------------------------------+
|1  |bdc3aa61-cf2e-4350-b0ff-9bda0bcba214|
|2  |79f9566a-b9fc-44bf-9903-c320f8beacad|
|3  |9f8ea8a8-97cd-44b0-a661-0453a198e5e7|
|4  |3dca3b34-5210-4aaf-82e4-d39a3be94cc4|
|5  |65d97e4d-21f2-449f-9374-0ca22d86400b|
|6  |cee5a7ec-1962-40ed-956d-67b87616a800|
|7  |9a5088bc-21f3-4e17-b1a4-8877fc2ebc9c|
|8  |c3a1d801-35f0-45fb-add6-cdae6b0d4ca2|
|9  |f9c8c91b-24db-48d9-aaee-32fb3e5cf644|
|10 |3535e360-9244-46f0-9e79-c472c8477df9|
+---+------------------------------------+
only showing top 10 rows



In [8]:
df.select("id").show(5)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



В последнем примере мы использовали метод select, который работает аналогично обычному SQL. Давайте посмотрим, что ещё такого есть

In [9]:
from pyspark.sql.functions import col
df.filter(col("id") > 50).show(5)

+---+--------------------+
| id|                uuid|
+---+--------------------+
| 51|b6048618-785e-47d...|
| 52|a71f3d74-e6f5-454...|
| 53|5f22b1e3-66af-41c...|
| 54|f8a9967f-9d23-478...|
| 55|03b03dee-6077-466...|
+---+--------------------+
only showing top 5 rows



In [10]:
df.filter("id > 25").show(5)

+---+--------------------+
| id|                uuid|
+---+--------------------+
| 26|aa981509-53b9-426...|
| 27|63193047-2ee6-410...|
| 28|eff9431a-0fd4-44f...|
| 29|6b69a55b-3f3a-4fe...|
| 30|38fe7509-7672-4c8...|
+---+--------------------+
only showing top 5 rows



In [11]:
df.where("id > 40").show(5)

+---+--------------------+
| id|                uuid|
+---+--------------------+
| 41|db256541-103c-45c...|
| 42|d1dd218b-520e-462...|
| 43|f4b2fcaa-3a2b-49d...|
| 44|58c5ad92-afe8-49d...|
| 45|a6db1db9-7cb7-4b5...|
+---+--------------------+
only showing top 5 rows



In [12]:
df.drop("id").show(5)

+--------------------+
|                uuid|
+--------------------+
|bdc3aa61-cf2e-435...|
|79f9566a-b9fc-44b...|
|9f8ea8a8-97cd-44b...|
|3dca3b34-5210-4aa...|
|65d97e4d-21f2-449...|
+--------------------+
only showing top 5 rows



In [13]:
df.count()

100

In [14]:
df.filter(col("id") > 70).count()

30

### Джойны

In [14]:
dict_df = spark.read.yt("//sys/spark/examples/example_dict")
dict_df.show(5)

+--------------------+-----+
|                uuid|count|
+--------------------+-----+
|bdc3aa61-cf2e-435...|    1|
|3dca3b34-5210-4aa...|    0|
|9a5088bc-21f3-4e1...|    9|
|3535e360-9244-46f...|    0|
|4c4ff81e-d5c1-4ec...|    9|
+--------------------+-----+
only showing top 5 rows



In [15]:
df.join(dict_df, ["uuid"])

DataFrame[uuid: string, id: bigint, count: bigint]

Результат метода join -- это снова датафрейм, его можно положить в переменную, вызывать на нем методы filter, select, show, count и т п

In [16]:
df.join(dict_df, ["uuid"]).show()

+--------------------+---+-----+
|                uuid| id|count|
+--------------------+---+-----+
|bdc3aa61-cf2e-435...|  1|    1|
|3dca3b34-5210-4aa...|  4|    0|
|9a5088bc-21f3-4e1...|  7|    9|
|3535e360-9244-46f...| 10|    0|
|4c4ff81e-d5c1-4ec...| 13|    9|
|48b89381-4eaa-49d...| 16|    1|
|ba33e170-32a9-459...| 19|    1|
|3e7db34a-0f80-412...| 22|    6|
|ffc98b47-ae72-46c...| 25|    6|
|eff9431a-0fd4-44f...| 28|    5|
|9d7a4383-9494-4ce...| 31|    7|
|f9dafada-47f9-4a3...| 34|    0|
|882e9b21-1618-401...| 37|    3|
|b66a6085-56b0-46d...| 40|    0|
|f4b2fcaa-3a2b-49d...| 43|    9|
|0dd51b59-3a34-4d8...| 46|    1|
|956bd688-8c88-49b...| 49|    2|
|a71f3d74-e6f5-454...| 52|    6|
|03b03dee-6077-466...| 55|    8|
|66b01fb0-0598-483...| 58|    8|
+--------------------+---+-----+
only showing top 20 rows



In [17]:
df.join(dict_df, ["uuid"]).count()

34

In [18]:
df.join(dict_df, ["uuid"], "left_outer").show()

+--------------------+---+-----+
|                uuid| id|count|
+--------------------+---+-----+
|bdc3aa61-cf2e-435...|  1|    1|
|79f9566a-b9fc-44b...|  2| null|
|9f8ea8a8-97cd-44b...|  3| null|
|3dca3b34-5210-4aa...|  4|    0|
|65d97e4d-21f2-449...|  5| null|
|cee5a7ec-1962-40e...|  6| null|
|9a5088bc-21f3-4e1...|  7|    9|
|c3a1d801-35f0-45f...|  8| null|
|f9c8c91b-24db-48d...|  9| null|
|3535e360-9244-46f...| 10|    0|
|2e5c9611-a228-4e8...| 11| null|
|6efca41f-7b13-4f1...| 12| null|
|4c4ff81e-d5c1-4ec...| 13|    9|
|7e3368f3-64cf-4af...| 14| null|
|3e250454-5a3d-473...| 15| null|
|48b89381-4eaa-49d...| 16|    1|
|3fbeebb5-e13e-412...| 17| null|
|68ffe6c3-2096-43f...| 18| null|
|ba33e170-32a9-459...| 19|    1|
|4a192a1e-d79e-459...| 20| null|
+--------------------+---+-----+
only showing top 20 rows



In [13]:
df.join(dict_df, ["uuid"], "left_outer").count()

100

In [14]:
df.filter(col("id") > 50).join(dict_df, ["uuid"], "left_outer").count()

NameError: name 'col' is not defined

### Запись в YT

In [18]:
df.distinct().write.yt("//home/sashbel/data/test_write")

По умолчанию, если по этому пути уже что-то есть, будет ошибка

In [23]:
df.write.yt("//home/sashbel/data/test_write")

AnalysisException: 'path yt:/home/sashbel/data/test_write already exists.;'

Если действительно хочется перезаписать таблицу, можно указать опцию overwrite

In [16]:
df.write.mode("overwrite").yt("//home/sashbel/data/test_write")

In [25]:
df.write.mode("overwrite").optimize_for("scan").yt("//home/sashbel/data/test_write")

Датафреймы можно сортировать и записывать отсортированный результат

In [26]:
df.sort("uuid").show()

+---+--------------------+
| id|                uuid|
+---+--------------------+
| 89|0192430d-d64f-4e3...|
| 62|02291f53-0fd4-476...|
| 32|02eb07c5-bb28-4de...|
| 82|02f1fde8-3c6f-48a...|
| 55|03b03dee-6077-466...|
| 46|0dd51b59-3a34-4d8...|
| 97|1e9f258d-27f2-43e...|
| 80|1fd27ff9-6cba-41d...|
| 70|23701b4e-20ae-430...|
| 35|2877626b-582f-4ad...|
| 67|289f5cc5-4286-432...|
| 11|2e5c9611-a228-4e8...|
| 93|31c88419-4d1f-4c6...|
| 10|3535e360-9244-46f...|
| 30|38fe7509-7672-4c8...|
| 95|3a94e7f9-bcd4-413...|
|  4|3dca3b34-5210-4aa...|
| 15|3e250454-5a3d-473...|
| 22|3e7db34a-0f80-412...|
| 65|3f406548-f649-4cf...|
+---+--------------------+
only showing top 20 rows



In [27]:
df.sort("uuid").coalesce(1).write.sorted_by("uuid").mode("overwrite").yt("//home/sashbel/data/test_write_sorted")

### UDF

In [9]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

def get_uuid_head(uuid):
    return uuid.split("-")[0]

get_uuid_head_udf = udf(get_uuid_head, StringType())

In [10]:
df.withColumn("uuid_head", get_uuid_head_udf(col("uuid"))).show()

+---+--------------------+---------+
| id|                uuid|uuid_head|
+---+--------------------+---------+
|  1|bdc3aa61-cf2e-435...| bdc3aa61|
|  2|79f9566a-b9fc-44b...| 79f9566a|
|  3|9f8ea8a8-97cd-44b...| 9f8ea8a8|
|  4|3dca3b34-5210-4aa...| 3dca3b34|
|  5|65d97e4d-21f2-449...| 65d97e4d|
|  6|cee5a7ec-1962-40e...| cee5a7ec|
|  7|9a5088bc-21f3-4e1...| 9a5088bc|
|  8|c3a1d801-35f0-45f...| c3a1d801|
|  9|f9c8c91b-24db-48d...| f9c8c91b|
| 10|3535e360-9244-46f...| 3535e360|
| 11|2e5c9611-a228-4e8...| 2e5c9611|
| 12|6efca41f-7b13-4f1...| 6efca41f|
| 13|4c4ff81e-d5c1-4ec...| 4c4ff81e|
| 14|7e3368f3-64cf-4af...| 7e3368f3|
| 15|3e250454-5a3d-473...| 3e250454|
| 16|48b89381-4eaa-49d...| 48b89381|
| 17|3fbeebb5-e13e-412...| 3fbeebb5|
| 18|68ffe6c3-2096-43f...| 68ffe6c3|
| 19|ba33e170-32a9-459...| ba33e170|
| 20|4a192a1e-d79e-459...| 4a192a1e|
+---+--------------------+---------+
only showing top 20 rows



In [11]:
get_uuid_tail_udf = udf(lambda uuid: uuid.split("-")[-1])
df.withColumn("uuid_tail", get_uuid_tail_udf(col("uuid"))).show()

+---+--------------------+------------+
| id|                uuid|   uuid_tail|
+---+--------------------+------------+
|  1|bdc3aa61-cf2e-435...|9bda0bcba214|
|  2|79f9566a-b9fc-44b...|c320f8beacad|
|  3|9f8ea8a8-97cd-44b...|0453a198e5e7|
|  4|3dca3b34-5210-4aa...|d39a3be94cc4|
|  5|65d97e4d-21f2-449...|0ca22d86400b|
|  6|cee5a7ec-1962-40e...|67b87616a800|
|  7|9a5088bc-21f3-4e1...|8877fc2ebc9c|
|  8|c3a1d801-35f0-45f...|cdae6b0d4ca2|
|  9|f9c8c91b-24db-48d...|32fb3e5cf644|
| 10|3535e360-9244-46f...|c472c8477df9|
| 11|2e5c9611-a228-4e8...|5d377ae484e2|
| 12|6efca41f-7b13-4f1...|823eb5f68ca2|
| 13|4c4ff81e-d5c1-4ec...|17bb5d6bdbab|
| 14|7e3368f3-64cf-4af...|a37fc49a5fe1|
| 15|3e250454-5a3d-473...|6ebf62b9f0a5|
| 16|48b89381-4eaa-49d...|088ae376b3b3|
| 17|3fbeebb5-e13e-412...|d464428e1c9a|
| 18|68ffe6c3-2096-43f...|57cd47092fba|
| 19|ba33e170-32a9-459...|c9224d4c1c16|
| 20|4a192a1e-d79e-459...|b99929c3b4d7|
+---+--------------------+------------+
only showing top 20 rows



### Агрегации

In [31]:
with_count = df.join(dict_df, ["uuid"], "left_outer")

In [32]:
with_count.show(5)

+--------------------+---+-----+
|                uuid| id|count|
+--------------------+---+-----+
|bdc3aa61-cf2e-435...|  1|    1|
|79f9566a-b9fc-44b...|  2| null|
|9f8ea8a8-97cd-44b...|  3| null|
|3dca3b34-5210-4aa...|  4|    0|
|65d97e4d-21f2-449...|  5| null|
+--------------------+---+-----+
only showing top 5 rows



In [33]:
with_count.filter(col("count") == 0).show(5)

+--------------------+---+-----+
|                uuid| id|count|
+--------------------+---+-----+
|3dca3b34-5210-4aa...|  4|    0|
|3535e360-9244-46f...| 10|    0|
|f9dafada-47f9-4a3...| 34|    0|
|b66a6085-56b0-46d...| 40|    0|
|7abb4ccb-1a6b-4ef...| 73|    0|
+--------------------+---+-----+



In [34]:
import pyspark.sql.functions as F
with_count.groupBy("count").agg(F.max("id").alias("max_id")).show()

+-----+------+
|count|max_id|
+-----+------+
|    0|    73|
|    7|    31|
| null|    99|
|    6|    82|
|    9|    94|
|    5|    85|
|    1|    46|
|    3|    97|
|    8|    88|
|    2|    49|
|    4|   100|
+-----+------+



In [35]:
with_count.filter(col("count").isNull()).sort(F.desc("id")).show(5)

+--------------------+---+-----+
|                uuid| id|count|
+--------------------+---+-----+
|a2d281b9-b44d-40b...| 99| null|
|a2256ca3-a972-4a3...| 98| null|
|aa431726-fb03-421...| 96| null|
|3a94e7f9-bcd4-413...| 95| null|
|31c88419-4d1f-4c6...| 93| null|
+--------------------+---+-----+
only showing top 5 rows



### Кастомные map и filter

In [36]:
df.filter(df.uuid.contains("aa")).show(5, truncate=False)

+---+------------------------------------+
|id |uuid                                |
+---+------------------------------------+
|1  |bdc3aa61-cf2e-4350-b0ff-9bda0bcba214|
|4  |3dca3b34-5210-4aaf-82e4-d39a3be94cc4|
|9  |f9c8c91b-24db-48d9-aaee-32fb3e5cf644|
|16 |48b89381-4eaa-49d3-b627-088ae376b3b3|
|21 |88744b37-aab7-4d17-a4cd-e9a52eaf8f03|
+---+------------------------------------+
only showing top 5 rows



In [37]:
from pyspark.sql.types import Row
df.rdd.map(lambda x: Row(*x.uuid.split("-"))).toDF().show(5, truncate=False)

+--------+----+----+----+------------+
|_1      |_2  |_3  |_4  |_5          |
+--------+----+----+----+------------+
|bdc3aa61|cf2e|4350|b0ff|9bda0bcba214|
|79f9566a|b9fc|44bf|9903|c320f8beacad|
|9f8ea8a8|97cd|44b0|a661|0453a198e5e7|
|3dca3b34|5210|4aaf|82e4|d39a3be94cc4|
|65d97e4d|21f2|449f|9374|0ca22d86400b|
+--------+----+----+----+------------+
only showing top 5 rows



### Конвертация в Pandas

Спарковый датафрейм можно превратить в Pandas, и продолжить работу с ним. **Внимание**: при этом все данные соберутся в память на локальной машине (той, где ноутбук). Обычно делают так: берут много данных, агрегируют их на Spark, потом маленький агрегат собирают в Pandas, и строят график, например 

In [38]:
import pyspark.sql.functions as F
aggregate = df.withColumn("id_mod_10", F.col("id") % 10).groupBy("id_mod_10").agg(F.sum("id"))
aggregate.show()

+---------+-------+
|id_mod_10|sum(id)|
+---------+-------+
|        0|    550|
|        7|    520|
|        6|    510|
|        9|    540|
|        5|    500|
|        1|    460|
|        3|    480|
|        8|    530|
|        2|    470|
|        4|    490|
+---------+-------+



In [39]:
pd_df = aggregate.toPandas()
pd_df

,id_mod_10,sum(id)
0,0,550
1,7,520
2,6,510
3,9,540
4,5,500
5,1,460
6,3,480
7,8,530
8,2,470
9,4,490


### Колонки с YSON

В Spark у датафрейма могут быть колонки со сложной схемой: списки, мапы, именованные структуры. В YT такие колонки часто записаны с типом Any, а в значении лежит Yson. Чтобы Spark смог такую колонку правильно прочитать, ему надо подсказать, что там на самом деле лежит. Иначе он выдаст вам кучу байт :(

In [6]:
df_yson = spark.read.yt("//sys/spark/examples/example_yson")

In [13]:
df_yson.collect()

[Row(value=bytearray(b'{\x01\x02a=\x02\x02;\x01\x02b=\x02\x04}')),
 Row(value=bytearray(b'{\x01\x02c=\x02\x06}'))]

In [8]:
from pyspark.sql.types import MapType, StringType, LongType
df_yson_map = spark.read \
    .schema_hint({
        "value": MapType(StringType(), LongType())
    }) \
    .yt("//sys/spark/examples/example_yson")
df_yson_map.show()

+----------------+
|           value|
+----------------+
|[b -> 2, a -> 1]|
|        [c -> 3]|
+----------------+



In [9]:
df_yson_struct = spark.read \
    .schema_hint({
        "value": {"a": LongType(), "b": LongType(), "c": LongType()}
    }) \
    .yt("//sys/spark/examples/example_yson")
df_yson_struct.show()

+-------+
|  value|
+-------+
|[1, 2,]|
| [,, 3]|
+-------+



In [10]:
df_yson_struct.select("value.a").show()

+----+
|   a|
+----+
|   1|
|null|
+----+



In [11]:
df_yson_struct.select("value.b").show()

+----+
|   b|
+----+
|   2|
|null|
+----+



In [12]:
df_yson_struct.select("value.c").show()

+----+
|   c|
+----+
|null|
|   3|
+----+



### BinaryType vs YsonType vs StringType

In [33]:
import yt.wrapper as yt
from yt.wrapper import YtClient
def get_schema(path):
    return yt.get("{}/@schema".format(path), client=YtClient(proxy="hume"))

Если вы читаете из YT колонку с типом Any без хинтов, в Spark у неё будет тип YsonType

In [18]:
yson_df = spark.read.yt("//sys/spark/examples/example_yson")
yson_df.printSchema()

root
 |-- value: yson (nullable = true)



В collect, take и т п он преобразуется в bytearray

In [19]:
yson_df.take(1)

[Row(value=bytearray(b'{\x01\x02a=\x02\x02;\x01\x02b=\x02\x04}'))]

Его можно явно прикастовать к BinaryType в Spark

In [29]:
binary_yson_df = yson_df.withColumn("value", f.col("value").cast(t.BinaryType()))

In [30]:
binary_yson_df.printSchema()

root
 |-- value: binary (nullable = true)



In [31]:
binary_yson_df.take(1)

[Row(value=bytearray(b'{\x01\x02a=\x02\x02;\x01\x02b=\x02\x04}'))]

При записи YsonType в YT получится колонка с типом Any

In [35]:
yson_df.write.mode("overwrite").yt("//sys/spark/examples/example_yson_2")
get_schema("//sys/spark/examples/example_yson_2")

{'value': [{'type_v3': {'type_name': 'optional', 'item': 'yson'}, 'type': 'any', 'required': false, 'type_v2': {'metatype': 'optional', 'element': 'any'}, 'name': 'value'}], 'attributes': {'strict': true, 'unique_keys': false}}

При записи колонки с типом BinaryType в YT получается колонка с типом String

In [36]:
binary_yson_df.write.mode("overwrite").yt("//sys/spark/examples/example_yson_3")
get_schema("//sys/spark/examples/example_yson_3")

{'value': [{'type_v3': {'type_name': 'optional', 'item': 'string'}, 'type': 'string', 'required': false, 'type_v2': {'metatype': 'optional', 'element': 'string'}, 'name': 'value'}], 'attributes': {'strict': true, 'unique_keys': false}}

Если вы читаете колонку с типом String, по умолчанию Spark считает, что это StringType.
Если там лежат байты, нужен хинт

In [37]:
binary_df = spark.read.schema_hint({"value": t.BinaryType()}).yt("//sys/spark/examples/example_yson_3")

In [38]:
binary_df.printSchema()

root
 |-- value: binary (nullable = true)



In [39]:
binary_df.show()

+--------------------+
|               value|
+--------------------+
|[7B 01 02 61 3D 0...|
|[7B 01 02 63 3D 0...|
+--------------------+



Если у вас есть колонка BinaryType и вы знаете, что там Yson и её надо записать в YT как Any, то надо кастануть её к YsonType

In [41]:
from spyt.types import YsonType
binary_df\
  .withColumn("value", f.col("value").cast(YsonType()))\
  .write.mode("overwrite").yt("//sys/spark/examples/example_yson_4")
get_schema("//sys/spark/examples/example_yson_4")

{'value': [{'type_v3': {'type_name': 'optional', 'item': 'yson'}, 'type': 'any', 'required': false, 'type_v2': {'metatype': 'optional', 'element': 'any'}, 'name': 'value'}], 'attributes': {'strict': true, 'unique_keys': false}}

Но если там не Yson, то запись упадёт с ошибкой

Итого:
    
    
    читаем Any -> YsonType 
    читаем String -> StringType
    читаем String с хинтом -> BinaryType
    
    YsonType можно кастануть к BinaryType
    BinaryType можно кастануть к YsonType, будет доп валидация
    
    YsonType -> пишем Any
    BinaryType -> пишем String
    StringType -> пишем String
    

### UInt64

Беззнаковое 64-битное число реализовано отдельным типом

In [23]:
import spyt
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, LongType
import spyt.types as ST

In [24]:
spark = spyt.connect(yt_proxy='hahn', discovery_path="//home/dev/alex-shishkin/spark-test-11")
df = spark.read.yt("//sys/spark/examples/example_1").select("id")
df.printSchema()
df.show()

2021-09-14 11:47:15,915 - WARNING - spyt.client - SparkSession already exists and will be reused. Some configurations may not be applied. You can use spyt.stop(spark) method to close previous session
2021-09-14 11:47:15,944 - INFO - spyt.client - SPYT Cluster version: 3.0.1-1.14.3~beta1-alex-shishkin+yandex
2021-09-14 11:47:15,946 - INFO - spyt.client - SPYT library version: 1.14.6-SNAPSHOT
2021-09-14 11:47:16,167 - INFO - spyt.client - SHS link: http://sas0-1758-node-hahn.sas.yp-c.yandex.net:27001/history/app-20210914111402-0000/jobs/


root
 |-- id: long (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
+---+
only showing top 20 rows



Преобразование long в UInt64 происходит с помощью каста

In [25]:
uint64_df = df.withColumn("id", F.col("id").cast(ST.UInt64Type()))
uint64_df.printSchema()
uint64_df.show(5)

root
 |-- id: uint64 (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



Выполнять арифметические операции напрямую нельзя, если это необходимо, нужно кастануть к long, а после подсчётов кастануть обратно к UInt64.

Преобразование UInt64 в long происходит с помощью каста

In [26]:
long_df = uint64_df.withColumn("id", F.col("id").cast(LongType()))
long_df.printSchema()
long_df.show(5)

root
 |-- id: long (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



Запись и чтение UInt64

In [27]:
uint64_df.write.mode("overwrite").yt("//sys/spark/examples/example_uint64")
read_uint64_df = spark.read.yt("//sys/spark/examples/example_uint64")
read_uint64_df.printSchema()
read_uint64_df.show(5)

root
 |-- id: uint64 (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



Если планируется использовать колонку как неизменяемый id, то удобно пользоваться строками.

Преобразование UIn64 в строчку происходит с помощью uint64_to_string_udf

In [28]:
string_df = uint64_df.withColumn("id", ST.uint64_to_string_udf(col("id")))
string_df.printSchema()
string_df.show(5)

root
 |-- id: string (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



Преобразование строчки в UInt64 происходит с помощью string_to_uint64_udf

In [29]:
uint64_df = string_df.withColumn("id", ST.string_to_uint64_udf(col("id")))
uint64_df.printSchema()
uint64_df.show(5)

root
 |-- id: uint64 (nullable = true)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows

